<a href="https://colab.research.google.com/github/ers6/writing-group-survey-interpreting-tool/blob/main/Writing_Group_Survey_Interpretation_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Writing Group Survey Interpretation Tool

---
This google colab notebook interprets results of the Writers Workshops' [Writing Group Survey](https://forms.gle/XuDknUBCjzhWChbX9) 

Prior to running this program: 
1. Download the survey results google sheet as a .csv file. This is a format that python can read more easily than a google sheet. 
2. Create a blank csv file on your computer. This program will automatically populate that file with the groups it creates, contact information for people in the group, and a template email to send each member.  

To run this program, press your shift and return keys at the same time or press the play button on each cell. It is important to run the notebook in the order that it is written. You will know a cell has been run when a small green check mark appears on the left hand side of it

After the program finishes running, access the blank csv file you created. Each row in the file corresponds to a writing group, their members, and an email to send to them. For people who could not be matched, the file contains rows of their names, contact information, and an email to send them as well. 

Before sending emails, be sure to review the safety concerns report which appears in the last cell of the program. 

## Importing Python Libraries 

In [ ]:
import pandas as pd
import numpy as np
import csv
import random
import re
from google.colab import files
import io

## Importing the survey results file and outfile
1. Upload the csv file of the survey results you created to colab 
2. locate the file in the left hand tab by clicking on the file icon (hint-if you can't find it, it's probably in the content folder).
3. Right click the file you uploaded. Select "copy path" 
4. when prompted to input the results file path, paste the file path from your clipboard. 
5. Upload the csv file where the results will be stored (outfile)
6. locate the file in the left hand tab by clicking on the file icon. 
7. Right click the outfile you uploaded. Select "copy path"
8. When prompted to input the out file path, paste the file path from your clipboard. 

In [ ]:
uploaded = files.upload()

Saving Writing Support Survey - Pilot Program v2 (Responses) - Form Responses 1.csv to Writing Support Survey - Pilot Program v2 (Responses) - Form Responses 1.csv


In [ ]:
results_file_path = input('input the path to the results file: ')

input the path to the results file: /content/Writing Support Survey - Pilot Program v2 (Responses) - Form Responses 1.csv


In [ ]:
outfile = files.upload()

Saving emails-to-send.csv to emails-to-send.csv


In [ ]:
out_file_path = input('input the path to the out file: ')

input the path to the out file: /content/emails-to-send.csv


## Setting up some functions we will use to analyze the results 
This code shouldn't do anything when it is run but it is important to run all these cells before proceeding to the next section

In [ ]:
def csv_to_dict(csv_file_name):
    results = []
    with open(csv_file_name, 'r', newline='', encoding='utf-8') as infile:
        csvin = csv.reader(infile)
        headers = next(csvin)
        # Make headers str.lower
        headers = [header.strip().lower() for header in headers]
        # Save dictionary of header:value for each row of data
        for row in csvin:
            n = 0
            your_dict = {}
            for column in row:
                your_dict[headers[n]] = column
                n += 1
            results.append(your_dict)
    return results

def makes_results_csv(results, outfile_name):
    headers = results[0].keys()
    rows = results
    print(rows)
    with open(outfile_name, 'w', encoding='UTF-8', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=headers)
        writer.writeheader()
        for row in rows:
            try:
                writer.writerow(row)
            except AttributeError:
                pass

In [ ]:
# note that this function assumes that the survey questions will be phrased exactly as they are in the response dictionary keys
# if you change the survey questions and schema without altering this function, the program will crash. 

def crosswalks_results(response):
    a_response = {'synchronous': False,
 'modality': 'online', 
 'sesh_length': 0, 
 'sesh_per_week': 0,
  'illinois_email' : '', 
  'discord_handle': '', 
  'safety_concern': ''
}
    if response['would you like to work synchronously (meeting with other writers at the same time) or asynchronously (communicating progress via discord)?'] == 'synchronously':
        a_response['synchronous'] = True
    if response['would you like to meet in person or online?'] == 'in person':
        a_response['modality'] = 'in_person'
    if response['how long would you like sessions to last (ideally)?'] == '':
        a_response['sesh_length'] = np.nan
    else:
        try:
            a_response['sesh_length'] = int(response['how long would you like sessions to last (ideally)?'].split()[0])
        except AttributeError: 
            a_response['sesh_length'] = 1.5
        except ValueError: 
            a_response['sesh_length'] = 1.5
    if response['how many times a week would you like to meet?'] == '':
        a_response['sesh_per_week'] = np.nan
    else:
        a_response['sesh_per_week'] = response['how many times a week would you like to meet?']
    a_response['illinois_email'] = response['please provide your university of illinois email address. we will use this to ensure you are affiliated with the university.']
    a_response['discord_handle'] = response["please provide your discord handle. we will use this to put group members in contact  with each other. if you don't have a discord account follow this guide to create one (it's completely free!): \nhttps://support.discord.com/hc/en-us/articles/360033931551-getting-started"]
    a_response['safety_concern'] = response["your safety is important. is there anyone with whom you would feel unsafe being in a group with? this information will be kept confidential. please only provide their name; no need to disclose the reason you'd feel unsafe."]
    return a_response
        
    

In [ ]:
def safety_concerns_report(responses): 
    concerns = []
    for this_response in responses: 
        if str(this_response['safety_concern']).lower() == 'no':
            pass
        elif str(this_response['safety_concern']).lower() == 'n/a':
            pass
        elif this_response['safety_concern'] != '':
            concerns.append(this_response)
        else: pass 
    if len(concerns) > 0:
        print('Review these safety concerns:')
        for concern in concerns: 
            print(concern['illinois_email'] + ':')
            print(concern['safety_concern'])
            print(' ')

## Reading in the survey results
These cells read in the survey results from the file and format them prior to analysis

In [ ]:
raw_results = csv_to_dict(results_file_path)

In [ ]:

results = []

for response in raw_results: 
  results.append(crosswalks_results(response))

{'timestamp': '4/21/2023 15:41:32', 'would you like to work synchronously (meeting with other writers at the same time) or asynchronously (communicating progress via discord)?': 'synchronously', 'would you like to meet in person or online?': 'online', 'how long would you like sessions to last (ideally)?': '1 hour', 'how many times a week would you like to meet?': '3', 'please provide your university of illinois email address. we will use this to ensure you are affiliated with the university.': 'morris25@illinois.edu', "please provide your discord handle. we will use this to put group members in contact  with each other. if you don't have a discord account follow this guide to create one (it's completely free!): \nhttps://support.discord.com/hc/en-us/articles/360033931551-getting-started": 'himalutenthusiast (j/k)', "your safety is important. is there anyone with whom you would feel unsafe being in a group with? this information will be kept confidential. please only provide their name;

In [ ]:
d = list(results[0].keys())
df = pd.DataFrame(columns = list(results[0].keys()))
df = df.append(results, ignore_index= True)

df

<ipython-input-49-aadbf7c014dd>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(results, ignore_index= True)


,synchronous,modality,sesh_length,sesh_per_week,illinois_email,discord_handle,safety_concern
0,True,online,1.0,3,morris25@illinois.edu,himalutenthusiast (j/k),Ludwig von Beethoven
1,False,online,NaN,NaN,morris25@illinois.edu,Hiladelphia.com,Adolf Hitler
2,True,in_person,3.0,2,blafond2@illinois.edu,BriL#3034,NA
3,True,online,1.5,3,morris25@illinois.edu,cantstopwontstoplolololol,Donald Trump
4,True,online,3.0,1,sarahu2@illinois.edu,kodgnisao;hfgiso,NA
5,False,online,NaN,NaN,yournamehere@illinois.edu,asdgajsid'jnfgisod'hagji,NA
6,True,in_person,1.0,1,ag58@illinois.edu,N/A,N/A
7,True,in_person,1.0,2,zef2@illinois.edu,N/a,N/a
8,True,in_person,2.0,2,Test Submission,Test Submission,No
9,False,online,NaN,NaN,dae2@illinois.edu,None,


## Analyzing Survey Results

These cells analyze the results, create groups, and drafts emails to send to them.

In [ ]:
# dividing into 2 DF based on synchronous/asynchronous
synchronous = df.loc[df['synchronous']==True]
asynchronous= df.loc[df['synchronous']==False]

In [ ]:
#  dividing in 2 based on modality for synchronous sessions (in person vs. online)
synchronous_in_person = synchronous.loc[synchronous['modality'] == 'in_person']
synchronous_online = synchronous.loc[synchronous['modality'] == 'online']

In [ ]:
#  dividing synchronous online into 3 groups based on sessions/week
synch_on_1seshes = synchronous_online.loc[synchronous_online['sesh_per_week'] == 1.0]
synch_on_2seshes = synchronous_online.loc[synchronous_online['sesh_per_week'] == 2.0]
synch_on_3seshes = synchronous_online.loc[synchronous_online['sesh_per_week'] == 3.0]

on_dfs = [synch_on_1seshes, synch_on_2seshes, synch_on_3seshes]




In [ ]:
# didiving synchronous in person into 5 groups based on sessions/week 
synch_in_1seshes = synchronous_in_person.loc[synchronous_in_person['sesh_per_week'] == '1']
synch_in_2seshes = synchronous_in_person.loc[synchronous_in_person['sesh_per_week'] == '2']
synch_in_3seshes = synchronous_in_person.loc[synchronous_in_person['sesh_per_week'] == '3']

in_dfs = [synch_in_1seshes, synch_in_2seshes, synch_in_3seshes]


In [ ]:
#  creating a dictionary of the sorted dataframes
all_responses = {"in_person": in_dfs, "online": on_dfs, "asynchronous": asynchronous}

In [ ]:
#  collecting unmatched people
unmatched =[]
for df in all_responses['in_person']:
    if len(df) == 1:
        unmatched.append(df['illinois_email'])

for df in all_responses['online']:
    if len(df) == 1:
        unmatched.append(df['illinois_email'])



In [ ]:
# now we need to generate emails to tell people they have been matched into a group
all_groups = []
#  in person: 
for group in all_responses['in_person']: 
    
    try:
        contacts = {'emails': group['illinois_email'].tolist(),
               'discord_handles': group['discord_handle'].tolist(),
               'modality': group['modality'].tolist()[0],
                'sesh_per_week': group['sesh_per_week'].tolist()[0],
               'sesh_length': round(group['sesh_length'].mean())}
        all_groups.append(contacts)
    except ValueError: #  assume this means that no one selected this option so we will skip it 
        pass
    except IndexError: 
        pass
for group in all_responses['online']: 
    
    try:
        contacts = {'emails': group['illinois_email'].tolist(),
               'discord_handles': group['discord_handle'].tolist(),
               'modality': group['modality'].tolist()[0],
                'sesh_per_week': group['sesh_per_week'].tolist()[0],
               'sesh_length': round(group['sesh_length'].mean())}
        all_groups.append(contacts)
    except ValueError: #  assume this means that no one selected this option so we will skip it 
        pass
    except IndexError:
        pass 

    

## Sending Emails

In [ ]:
# now we need to generate emails to tell people they have been matched into a group

def make_contact_list(discord_handles):
    discords = ''
    i = 0
 
    for this_handle in discord_handles: 
        discords += this_handle + ', '

    return discords[:-2] 

def write_emails(group): 
    if len(group['emails']) == 1: 
        message = "Hello! We regret to inform you that we have been unable to find a writing group that overlaps with your preferences. Please see the Writers Workshop's website for information about our regularly scheduled groups, and sign up for an appointment with us for writing support. Thanks!"
    elif len(group['emails']) > 1:
        message = "Hello! We are writing to let you know that you have been matched into a writing group that will meet" + " " + re.sub('_', ' ',str(group['modality'])) + " " + str(group['sesh_per_week']) + " "+ "time(s) per week for" + " "+ str(group['sesh_length']) +" "+ "hour(s). Please coordinate with the members of your group via discord to arrange a time that works best for you. Their discord handles are:" +" "+make_contact_list(group['discord_handles']) +". Happy writing!"
    else: pass
    return message


In [ ]:
for group in all_groups:
    group['email_to_send'] = write_emails(group)

In [ ]:
all_groups

[{'emails': ['ag58@illinois.edu'],
  'discord_handles': ['randomdiscord9182'],
  'modality': 'in_person',
  'sesh_per_week': '1',
  'sesh_length': 1,
  'email_to_send': "Hello! We regret to inform you that we have been unable to find a writing group that overlaps with your preferences. Please see the Writers Workshop's website for information about our regularly scheduled groups, and sign up for an appointment with us for writing support. Thanks!"},
 {'emails': ['blafond2@illinois.edu', 'zef2@illinois.edu', 'Test Submission'],
  'discord_handles': ['randomdiscord3000',
   'randomdiscord3478',
   'randomdiscord8573'],
  'modality': 'in_person',
  'sesh_per_week': '2',
  'sesh_length': 2,
  'email_to_send': 'Hello! We are writing to let you know that you have been matched into a writing group that will meet in person 2 time(s) per week for 2 hour(s). Please coordinate with the members of your group via discord to arrange a time that works best for you. Their discord handles are: randomdi

In [ ]:
makes_results_csv(all_groups, out_file_path)

[{'emails': ['ag58@illinois.edu'], 'discord_handles': ['randomdiscord9182'], 'modality': 'in_person', 'sesh_per_week': '1', 'sesh_length': 1, 'email_to_send': "Hello! We regret to inform you that we have been unable to find a writing group that overlaps with your preferences. Please see the Writers Workshop's website for information about our regularly scheduled groups, and sign up for an appointment with us for writing support. Thanks!"}, {'emails': ['blafond2@illinois.edu', 'zef2@illinois.edu', 'Test Submission'], 'discord_handles': ['randomdiscord3000', 'randomdiscord3478', 'randomdiscord8573'], 'modality': 'in_person', 'sesh_per_week': '2', 'sesh_length': 2, 'email_to_send': 'Hello! We are writing to let you know that you have been matched into a writing group that will meet in person 2 time(s) per week for 2 hour(s). Please coordinate with the members of your group via discord to arrange a time that works best for you. Their discord handles are: randomdiscord3000, randomdiscord347

In [ ]:
safety_concerns_report(results)


Review these safety concerns:
morris25@illinois.edu:
Ludwig von Beethoven
 
morris25@illinois.edu:
Adolf Hitler
 
blafond2@illinois.edu:
NA
 
morris25@illinois.edu:
Donald Trump
 
sarahu2@illinois.edu:
NA
 
yournamehere@illinois.edu:
NA
 
